## **Cle API**

In [1]:
!pip -q install -U langchain langchain-core langchain-community langgraph langchain-google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google

In [2]:
import os, getpass

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI (Gemini) API key: ")


Enter your Google AI (Gemini) API key: ··········


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # rapide/économique ; tu peux changer
    temperature=0
)


In [8]:
llm.invoke("Donne-moi une phrase de 4 mots").content


'Le chat dort bien.'

## **Creation des outils**

In [9]:
from langchain.tools import tool
import math

@tool
def calc(expression: str) -> str:
    """Évalue une expression mathématique simple. Ex: '2*(3+4)' ou 'sqrt(2)'."""
    allowed = {"sqrt": math.sqrt, "pi": math.pi, "e": math.e}
    try:
        # eval "restreint" : pas d'accès aux builtins
        result = eval(expression, {"__builtins__": {}}, allowed)
        return str(result)
    except Exception as e:
        return f"Erreur: {e}"

@tool
def faq(produit: str) -> str:
    """Retourne des infos internes sur un produit (exemple)."""
    base = {
        "alpha": "Alpha: abonnement mensuel, support email.",
        "beta": "Beta: paiement à l'usage, support chat.",
    }
    return base.get(produit.lower(), "Je n'ai rien sur ce produit.")


## **Construction de l'agent**

In [10]:
from langgraph.prebuilt import create_react_agent

tools = [calc, faq]

agent = create_react_agent(
    llm,
    tools=tools,
    prompt="Tu es un assistant utile. Utilise les outils quand c'est pertinent."
)


/tmp/ipython-input-1273727277.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


## **Premiere requete**

In [16]:
inputs = {"messages": [{"role": "user", "content": "quelle est l'heure actuelle en france en 4 mots"}]}

result = agent.invoke(inputs)
result["messages"][-1].content


[{'type': 'text',
  'text': "Je ne peux pas vous donner l'heure.",
  'extras': {'signature': 'CoADAXLI2nzHsdhlE1fvD70V3i+fSwQQjk3JKMHe7/W2AbSEcolqXMIfXpNTK7DUkASEVkqvqhk87qwMiOUG4E5aH0lel7kCSJyr8O9TVIftlt2ZtW4WGjgMC2u9tq03W53G++O+CmNsvGBHjjBly/YxLl7lmRkV6XdVbL1SNYDYTvWSd/Y4FCc9Xl5HRx52aius9Zp69zSVAm2w0n9jZeaX87SsGHrR8ylmqv84CM9iIaSQCsZNGHLx/9cLZcJK+8IiXcE11IQWkEuAXUix2GBqXVxsBVE6KrrLnt06Dq1u63C05WnOFnzn0Qxr3eGlZ+7nAuzHBCImG/ZIn86j+rA4VPm8TLBrweDlUedHMW/849nIYuiNo74nH+BBCNr8WI5gvGWj8qkcx8JnsmVJIkL+rGGAoQTX+AXkDX1WEEXU+S1L3r0GKFIfQ6lFP7LTUM7R0SjTyQfs2q0ws7yqaF3ekuqVvlldDmnn+uDCL7arW5ot+MDBjd4fP6jCzrW49cSW'}}]

## **suivie de l'agent**

In [17]:
for chunk in agent.stream(inputs, stream_mode="updates"):
    print(chunk)


{'agent': {'messages': [AIMessage(content=[{'type': 'text', 'text': "Je ne peux pas vous donner l'heure.", 'extras': {'signature': 'CoADAXLI2nwEHNL/860FxsShLmvZqkWJtt+jvjvYH7bcmkQvtsaCrvxqcuq3pdberUb0OTpSRJA2Yvknxd5T3CxAdMtF1xcXVcMTFMnAHftiSimDt6FOHOJz61joSpPiMiCFdPQMHtdCnUTA6LeNGZBmTlmg6SzisVgHi1E1DfmLdxIEs1dP4LF6OIoNZJJ+QvFIvjVl9LwALtCqBWyfnUDUJPFPN4QqvJ3wKsUZLwP/Z6Fyc6pULhD9ABTScSKVi20ILmujnrzMmemdRAhi2HgteakeLo9OgwIJgws/e1JZaolTLZYhXsWpD3aRV/VHMmgv7KUsgKlqZKNFx5eBb77RgIX9ttFW8X+6YlInYg5mJ8+FF41L/kEHaPR0pwLifKTtBw1RSu3mxcp2paebuDQvUqagV6u+JHxyLGsuK52jbwBTUiLTKUW9lj7CxXACSd+GSXyOWDIeb/ev6Gv/4PeKzjZ1NUc8E3TlF6tAZOQana5Rgh3gLCovK2Y+2VgkjpAP'}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b98a6-8291-7203-9600-47a65db7bdea-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 126, 'output_tokens': 75, 'total_tokens': 201, 'input_token_detail